In [1]:
import pandas as pd
import numpy as np
from functions import *
import random
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

remove_fight_island = False

In [2]:
#Load models
#REMINDER: We are going to need to use 'eval' to get the models usable
with open('../data/production_models.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
print(len(models))


5


In [3]:
models

[['model_06_13_20',
  'model_06_20_20',
  'model_6_27_20',
  'model_7_18_20',
  'model_7_25_20',
  'model_9_2_20',
  'model_9_10_20',
  'model_10_23_20',
  'model_10_27_20',
  'model_10_28_20',
  'model_2021_06_24',
  'model_2021_06_30',
  'model_2021_07_21'],
 ['LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)',
  'GaussianNB()',
  'GaussianNB()',
  'GradientBoostingClassifier(n_estimators=122, random_state=75)',
  'GaussianNB()',
  'GaussianNB(var_smoothing=1e-12)',
  'GaussianNB(var_smoothing=1e-12)',
  'GradientBoostingClassifier(max_depth=95, min_samples_leaf=6, n_estimators=195,\n                           random_state=75)',
  "RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=197,\n                       n_jobs=-1, random_state=75)",
  "RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=22,\n                       n_jobs=-1, random_state=75)",
  'GradientBoostingClassifier(max_depth=95, min_samples_leaf=0.0015,\n

In [4]:
df = pd.read_csv("../data/kaggle_data/ufc-master.csv")

In [5]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
df = df[(df['Winner'] == 'Blue') | (df['Winner'] == 'Red')]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[250:]
odds_train = odds_df[250:]
label_train = label_df[250:]

df_test = df[:250]
odds_test = odds_df[:250]
label_test = label_df[:250]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

250
250
250
4554
4554
4554


<ipython-input-5-d6c2a1ffdd58>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'][mask] = 0
<ipython-input-5-d6c2a1ffdd58>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'][mask] = 1


In [6]:
if remove_fight_island:
    ##Let's remove the Fight island contests and see how that affects score
    df_test_no_fight_island = df_test[(df_test['location'] != 'Abu Dhabi, Abu Dhabi, United Arab Emirates')]
    df_train_no_fight_island = df_train[(df_train['location'] != 'Abu Dhabi, Abu Dhabi, United Arab Emirates')]
    df_test = df_test_no_fight_island
    df_train = df_train_no_fight_island



print(len(df_test))
print(len(df_train))


250
4554


In [7]:
#Set a value for the nulls in the ranks

weightclass_list = ['B_match_weightclass_rank', 'R_match_weightclass_rank', "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank", "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 'B_Pound-for-Pound_rank']
df_train[weightclass_list] = df_train[weightclass_list].fillna(17)
df_test[weightclass_list] = df_test[weightclass_list].fillna(17)

C:\Users\matth\anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [8]:
len(models[0])

13

In [9]:
score_list = []
for model_num in range(len(models[0])):
    test_model_name = models[0][model_num]
    test_model = eval(models[1][model_num])
    test_model_features = eval(models[2][model_num])
    test_model_ev = eval(models[3][model_num])
    
    score_list.append(evaluate_model(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True))

(4554, 10)

(4554,)

(4554, 2)

(250, 10)

(250,)

...

(250, 19)

(250,)

(250, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,-210,175,1.000000,0.000000,1
1,-320,250,1.000000,0.000000,1
2,130,-155,1.000000,0.000000,0
3,-155,135,1.000000,0.000000,1
4,-240,195,0.222222,0.777778,0
...,...,...,...,...,...
245,175,-230,0.592593,0.407407,0
246,250,-305,1.000000,0.000000,0
247,-134,105,1.000000,0.000000,1
248,-105,-117,1.000000,0.000000,1



          Number of matches: 250
          Number of bets: 232
          Number of winning bets: 125
          Number of losing bets: 107
          Number of underdog bets: 127
          Number of underdog wins: 55
          Number of underdog losses: 72
          Number of Favorite bets: 102
          Number of favorite wins: 70
          Number of favorite losses: 32
          Number of even bets: 3
          Number of even wins: 0
          Number of even losses: 3
          Profit: 1302.4175931046816
          Profit per bet: 5.613868935796042
          Profit per match: 5.209670372418726
          
          


In [10]:
print(models[4])

['1127.19067842605', '-950.9775482241005', '1083.4788618676141', '407.0778237930258', '-676.5402515344442', '-528.4063020903642', '-230.17923849846807', '-1678.1779331752696', '-120.80773041188252', '-1853.1849933783474', '1360.7010445967346', '66.33685543598621', '2208.3026702916454']


In [11]:
print(score_list)

[951.5496527850246, -565.0992471802674, 497.235950790235, -364.28056393596614, -1098.8503562877127, -1139.6492131677433, -342.33124048493767, -1027.165161820317, -1639.5499674873943, 1497.0601934872413, 900.3856353060073, -80.32868471321603, 1302.4175931046816]


In [12]:
models[4] = score_list

In [13]:
models

[['model_06_13_20',
  'model_06_20_20',
  'model_6_27_20',
  'model_7_18_20',
  'model_7_25_20',
  'model_9_2_20',
  'model_9_10_20',
  'model_10_23_20',
  'model_10_27_20',
  'model_10_28_20',
  'model_2021_06_24',
  'model_2021_06_30',
  'model_2021_07_21'],
 ['LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)',
  'GaussianNB()',
  'GaussianNB()',
  'GradientBoostingClassifier(n_estimators=122, random_state=75)',
  'GaussianNB()',
  'GaussianNB(var_smoothing=1e-12)',
  'GaussianNB(var_smoothing=1e-12)',
  'GradientBoostingClassifier(max_depth=95, min_samples_leaf=6, n_estimators=195,\n                           random_state=75)',
  "RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=197,\n                       n_jobs=-1, random_state=75)",
  "RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=22,\n                       n_jobs=-1, random_state=75)",
  'GradientBoostingClassifier(max_depth=95, min_samples_leaf=0.0015,\n

In [14]:
with open('../data/production_models.csv', 'w' , newline='') as outfile:
    writer = csv.writer(outfile)
    for row in models:
        print("HI")
        writer.writerow(row)
    
outfile.close()

HI
HI
HI
HI
HI
